In [1]:
import clingo
from clingox.backend import SymbolicBackend

In [2]:
prg = """

#program base.

time(1..3).

observe(comp, verzug, 3) :- not occurs(a, fill, 1), not occurs(a, fill, 2).
observe(comp, verzug, 3) :- occurs(a, fill, A), observe(p, displaced, B), A<B.

observe(p, displaced, T+1) :- time(T+1), occurs(_, pickup, T).
observe(p, displaced, T+1) :- time(T+1), observe(p, displaced, T).

#program instance.

occurs(c, pickup, 1).
occurs(a, fill, 2).

#program extract.

happened(Who, What, When) :- occurs(Who, What, When).

#program exploration.

counted(Who, What, Count) :-
    happened(Who, What, _),
    Count = #count {T : happened(Who, What, T)}.

1 { counterfactual(Who, What, C) : C=0..Count } 1 :- counted(Who, What, Count).
factual(Who, What, Count-CounterFactuals) :- counted(Who, What, Count), counterfactual(Who, What, CounterFactuals).

included(Who, What) :- factual(Who, What, N), N>0.
excluded(Who, What) :- counterfactual(Who, What, N), N>0.

Count { occurs(Who, What, T): time(T) } Count :- factual(Who, What, Count).

#program exploration_fail.

:- not observe(comp, verzug, 3).

#program exploration_success.

:- observe(comp, verzug, 3).

"""

In [3]:
filter = """

:- occurs(Who, What, When), factual(Who, What, C).
:- factual(Who, What, C), included(Who, What).
:- counterfactual(Who, What, C), excluded(Who, What).

"""

In [4]:
show_extracted = """

#show happened/3.

"""
show_relevant = """

#defined factual/3.
#defined counterfactual/3.
#defined excluded/2.
#defined included/2.
#defined observe/3.
#defined occurs/3.

#show factual/3.
#show counterfactual/3.
#show excluded/2.
#show included/2.
#show observe/3.
#show occurs/3.

"""

In [5]:
def solve(ctl: clingo.Control, atoms=False, shown=True, complement=False):
    with ctl.solve(yield_=True) as handle:
        result = handle.get()
        last_model = handle.model()
        while not result.exhausted:
            last_model = handle.model()
            handle.resume()
            result = handle.get()
        print(result)
        answer_set = []
        if result.satisfiable:
            answer_set = last_model.symbols(atoms=atoms, shown=shown, complement=complement)
        print(answer_set)
    return answer_set

In [6]:
def solve_all(ctl: clingo.Control):
    with ctl.solve(yield_=True) as handle:
        answer_sets = []
        for m in handle:
            symbols = m.symbols(shown=True)
            print("Answer {}:".format(m.number), " ".join(map(str, sorted(symbols))))
            answer_sets.append(symbols)
        result = handle.get()
        print(result)
    return answer_sets


In [7]:
extract = clingo.Control(("--models", "0"))
extract.add("base", [], prg)
extract.add("base", [], show_extracted)
extract.ground([("base", ()), ("instance", ()), ("extract", ())])
extracted = solve(extract)

SAT
[happened(c,pickup,1), happened(a,fill,2)]


In [8]:
success = clingo.Control(("--models", "0", "--enum-mode", "cautious"))
success.add("base", [], prg)
success.add("base", [], show_relevant)
with SymbolicBackend(success.backend()) as symb_backend:
    for e in extracted:
        symb_backend.add_rule((e,))

success.ground([("base", ()), ("exploration", ()), ("exploration_success", ())])
print("Necessary for success:")
needed = solve(success)
print()

Necessary for success:
SAT
[counterfactual(a,fill,0), factual(a,fill,1), included(a,fill)]



<block>:12:1-18: info: no atoms over signature occur in program:
  excluded/2

<block>:13:1-18: info: no atoms over signature occur in program:
  included/2

<block>:11:1-24: info: no atoms over signature occur in program:
  counterfactual/3

<block>:10:1-17: info: no atoms over signature occur in program:
  factual/3

<block>:14:1-17: info: no atoms over signature occur in program:
  observe/3

<block>:15:1-16: info: no atoms over signature occur in program:
  occurs/3



In [9]:
fail = clingo.Control(("--models", "0", "--enum-mode", "cautious"))
fail.add("base", (), prg)
fail.add("base", (), show_relevant)
with SymbolicBackend(fail.backend()) as symb_backend:
    for e in extracted:
        symb_backend.add_rule(head=(e,))
    for n in needed:
        symb_backend.add_rule(neg_body=(n,))

fail.ground([("base", ()), ("exploration", ()), ("exploration_fail", ())])
print("Symptom of failure:")
forbidden = solve(fail)
print()

Symptom of failure:
SAT
[counterfactual(a,fill,0), factual(a,fill,1), included(a,fill), observe(comp,verzug,3)]



<block>:12:1-18: info: no atoms over signature occur in program:
  excluded/2

<block>:13:1-18: info: no atoms over signature occur in program:
  included/2

<block>:11:1-24: info: no atoms over signature occur in program:
  counterfactual/3

<block>:10:1-17: info: no atoms over signature occur in program:
  factual/3

<block>:14:1-17: info: no atoms over signature occur in program:
  observe/3

<block>:15:1-16: info: no atoms over signature occur in program:
  occurs/3



In [10]:
suspect = clingo.Control(("--models", "0", "--enum-mode", "brave"))
suspect.add("base", (), prg + show_relevant)
with SymbolicBackend(suspect.backend()) as symb_backend:
    for e in extracted:
        symb_backend.add_rule(head=(e,))
    for n in needed:
        symb_backend.add_rule(neg_body=(n,))
    for f in forbidden:
        if f not in needed:
            symb_backend.add_rule(pos_body=(f,))

suspect.ground([("base", ()), ("exploration", ()), ("exploration_success", ())])
print("Suspected to cause failure:")
suspected = solve(suspect, complement=True)
print()

Suspected to cause failure:
SAT
[counterfactual(a,fill,1), factual(a,fill,0), observe(comp,verzug,3), observe(p,displaced,2), observe(p,displaced,3), occurs(a,fill,3), occurs(c,pickup,1), occurs(c,pickup,2), excluded(a,fill)]



<block>:55:1-18: info: no atoms over signature occur in program:
  excluded/2

<block>:56:1-18: info: no atoms over signature occur in program:
  included/2

<block>:54:1-24: info: no atoms over signature occur in program:
  counterfactual/3

<block>:53:1-17: info: no atoms over signature occur in program:
  factual/3

<block>:57:1-17: info: no atoms over signature occur in program:
  observe/3

<block>:58:1-16: info: no atoms over signature occur in program:
  occurs/3



In [11]:
suspected = tuple(s for s in suspected if s.match('occurs',3) and s not in forbidden)
suspected

(Function('occurs', [Function('a', [], True), Function('fill', [], True), Number(3)], True),
 Function('occurs', [Function('c', [], True), Function('pickup', [], True), Number(1)], True),
 Function('occurs', [Function('c', [], True), Function('pickup', [], True), Number(2)], True))

In [12]:
suspicion = clingo.Control(("--models", "0"))
#suspicion.add("base", (), filter)
with SymbolicBackend(suspicion.backend()) as symb_backend:
    symb_backend.add_rule(suspected, choice=True)
suspicion.ground([("base",())])
suspicions = solve_all(suspicion)

Answer 1: 
Answer 2: occurs(c,pickup,1)
Answer 3: occurs(c,pickup,2)
Answer 4: occurs(c,pickup,1) occurs(c,pickup,2)
Answer 5: occurs(a,fill,3)
Answer 6: occurs(a,fill,3) occurs(c,pickup,2)
Answer 7: occurs(a,fill,3) occurs(c,pickup,1)
Answer 8: occurs(a,fill,3) occurs(c,pickup,1) occurs(c,pickup,2)
SAT


In [13]:
for sus in suspicions:
    indication = clingo.Control(("--models", "0"))
    indication.add("base", (), prg)
    indication.add("base", (), show_relevant)

    with SymbolicBackend(indication.backend()) as symb_backend:
        for e in extracted:
            symb_backend.add_rule(head=(e,))
        for n in needed:
            symb_backend.add_rule(neg_body=(n,))
        for f in forbidden:
            if f not in needed:
                symb_backend.add_rule(pos_body=(f,))
        for s in sus:
            symb_backend.add_rule(neg_body=(s,))
    indication.ground([("base", ()), ("exploration", ()), ("exploration_success", ())])
    print("Suspicion:", sus)
    strong_suspected = solve_all(indication)

Suspicion: []
Answer 1: excluded(c,pickup) included(a,fill) counterfactual(a,fill,0) counterfactual(c,pickup,1) factual(a,fill,1) factual(c,pickup,0) occurs(a,fill,2)
Answer 2: included(a,fill) included(c,pickup) counterfactual(a,fill,0) counterfactual(c,pickup,0) factual(a,fill,1) factual(c,pickup,1) occurs(a,fill,2) occurs(c,pickup,3)
Answer 3: excluded(c,pickup) included(a,fill) counterfactual(a,fill,0) counterfactual(c,pickup,1) factual(a,fill,1) factual(c,pickup,0) occurs(a,fill,1)
Answer 4: included(a,fill) included(c,pickup) counterfactual(a,fill,0) counterfactual(c,pickup,0) factual(a,fill,1) factual(c,pickup,1) occurs(a,fill,1) occurs(c,pickup,3)
SAT
Suspicion: [occurs(c,pickup,1)]
UNSAT
Suspicion: [occurs(c,pickup,2)]
UNSAT
Suspicion: [occurs(c,pickup,1), occurs(c,pickup,2)]
UNSAT
Suspicion: [occurs(a,fill,3)]
UNSAT
Suspicion: [occurs(a,fill,3), occurs(c,pickup,2)]
UNSAT
Suspicion: [occurs(a,fill,3), occurs(c,pickup,1)]
UNSAT
Suspicion: [occurs(a,fill,3), occurs(c,pickup,1), 

<block>:12:1-18: info: no atoms over signature occur in program:
  excluded/2

<block>:13:1-18: info: no atoms over signature occur in program:
  included/2

<block>:11:1-24: info: no atoms over signature occur in program:
  counterfactual/3

<block>:10:1-17: info: no atoms over signature occur in program:
  factual/3

<block>:14:1-17: info: no atoms over signature occur in program:
  observe/3

<block>:15:1-16: info: no atoms over signature occur in program:
  occurs/3

<block>:12:1-18: info: no atoms over signature occur in program:
  excluded/2

<block>:13:1-18: info: no atoms over signature occur in program:
  included/2

<block>:11:1-24: info: no atoms over signature occur in program:
  counterfactual/3

<block>:10:1-17: info: no atoms over signature occur in program:
  factual/3

<block>:14:1-17: info: no atoms over signature occur in program:
  observe/3

<block>:15:1-16: info: no atoms over signature occur in program:
  occurs/3

<block>:12:1-18: info: no atoms over signature o